# 1.6: Generate WEPP Target Variable via WEPPcloud API

**Objective:** To programmatically execute a WEPP simulation using the WEPPcloud web service, monitor its progress, and download the final raster of average annual sediment yield. This notebook completely replaces the manual workflow of downloading climate and soil data.

### 1. Setup and Configuration

* **Imports:** Import necessary libraries (`wepppy`, `pathlib`, `yaml`, etc.).
* **Load Config:** Load the `weppcloud` section from the main `config.yml` file.
* **Load AOI:** Read the study area shapefile specified in the configuration.

In [ ]:
import sys
import yaml
from pathlib import Path
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt

# Add the source directory to the system path for module imports
def find_project_root(marker='config.yml'):
    """Find the project root by searching upwards for a marker file."""
    path = Path.cwd().resolve()
    while path.parent != path:
        if (path / marker).exists():
            return path
        path = path.parent
    raise FileNotFoundError(f"Project root with marker '{marker}' not found.")

PROJECT_ROOT = find_project_root()
if str(PROJECT_ROOT / 'src') not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT / 'src'))

# Import custom modules
from src.utils.logging import setup_colored_logging
from src.data_acquisition.weppcloud_client import (
    submit_weppcloud_job,
    monitor_job_status,
    download_weppcloud_result
)
from src.plotting_utils import plot_raster_with_boundary

log = setup_colored_logging()

# Load configuration
CONFIG_PATH = PROJECT_ROOT / "config.yml"
with open(CONFIG_PATH) as f:
    config = yaml.safe_load(f)

W_CONFIG = config['weppcloud']
log.info("✅ Configuration loaded successfully.")

# Resolve paths from config
AOI_PATH = PROJECT_ROOT / W_CONFIG['aoi_path']
OUTPUT_DIR = PROJECT_ROOT / W_CONFIG['output_directory']

log.info(f"Study Area Shapefile: {AOI_PATH}")
log.info(f"Output Directory: {OUTPUT_DIR}")

### 2. Call `weppcloud_client.py` Functions


In [ ]:
job_id = None
job_successful = False
final_raster_path = None

try:
    # --- Step 1: Submit Job ---
    job_id = submit_weppcloud_job(
        aoi_path=AOI_PATH,
        project_name=W_CONFIG['project_name'],
        simulation_years=W_CONFIG['simulation_years']
    )

    # --- Step 2: Monitor Status ---
    if job_id:
        # Using a shorter poll interval for this demonstration
        job_successful = monitor_job_status(job_id, poll_interval_sec=30)
    else:
        raise RuntimeError("Failed to submit job; could not retrieve a valid Job ID.")

    # --- Step 3: Download Result ---
    if job_successful:
        final_raster_path = download_weppcloud_result(
            job_id=job_id,
            output_dir=OUTPUT_DIR
        )
    else:
        log.error("WEPPcloud job did not complete successfully. Skipping result download.")

except Exception as e:
    log.critical(f"An unexpected error occurred during the workflow: {e}", exc_info=True)

### 3. Post-Flight Validation

* **Verify File:** Check that the downloaded raster exists and is not empty.
* **Check CRS and Extent:** Use `rasterio` to confirm that the output raster's projection and grid are consistent with the project's source DEM.
* **Visualize:** Display the final raster on a map to confirm it covers the correct geographic area.

In [ ]:
log.info("--- Workflow Complete: Final Status ---")

if final_raster_path and final_raster_path.exists():
    log.info(f"✅ SUCCESS: The target variable raster has been generated and saved to:")
    log.info(f"   -> {final_raster_path}")

    # --- 1. Verify Raster Metadata ---
    log.info("--- Verifying Raster Properties ---")
    try:
        with rasterio.open(final_raster_path) as src:
            log.info(f"   - Raster CRS: {src.crs}")
            log.info(f"   - Raster Dimensions: {src.width}x{src.height}")
            log.info(f"   - Raster NoData Value: {src.nodata}")
            log.info("✅ Raster metadata is readable and valid.")
    except Exception as e:
        log.error(f"❌ Failed to read raster metadata: {e}")

    # --- 2. Visualize the Final Output ---
    log.info("--- Generating Final Visualization ---")

    # Load the study area boundary for the plot
    aoi_gdf = gpd.read_file(AOI_PATH)

    fig, ax = plt.subplots(figsize=(12, 12), dpi=150)

    plot_raster_with_boundary(
        raster_path=final_raster_path,
        boundary_gdf=aoi_gdf,
        ax=ax,
        cmap='viridis',
        title="WEPPcloud Average Annual Sediment Yield"
    )

    # Save the figure to the reports directory
    REPORTS_DIR = PROJECT_ROOT / config['project_paths']['reports']
    figure_path = REPORTS_DIR / "weppcloud_sediment_yield_map.png"
    fig.savefig(figure_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
    log.info(f"✅ Final map saved to: {figure_path}")

    plt.show()

else:
    log.error("❌ FAILURE: The workflow did not produce the final target raster.")
    if not job_successful:
        log.error("\t- Reason: The WEPPcloud simulation failed or was cancelled.")
    else:
        log.error("\t- Reason: The result download failed after a successful simulation.")